In [91]:
include("../../src/utilities.jl")
include("../../src/model_with_ribosomes_functions.jl")
include("../../src/canonical_two_stage_model_with_division_functions.jl")
using Catalyst, DifferentialEquations, Plots, Statistics

In [92]:
function simulate_cycle(model, p, t0, t_fin, T, species_0)
    tstops = [t0+T:T:t_fin;]
    
    prob = DiscreteProblem(model, species_0, (t0,t_fin), p)
    jump_prob = JumpProblem(model, prob, Direct(), save_positions=(false,false))
        
    condition(u,t,integrator) = t%27.5==0.0
    affect!(integrator) = integrator.u = Utilities.partition_species(integrator.u)
    cb = DiscreteCallback(condition,affect!)
    
    sol = solve(jump_prob, SSAStepper(), saveat=0.01, callback=cb, tstops=tstops)
    return sol
end

function simulate_stochastic_cycle(model, p, t0, t_fin, T, species_0, N)
   prob = DiscreteProblem(
        model,
        species_0,
        (t0,t_fin),
        p
    )
    jump_prob = JumpProblem(model, prob, Direct())
    
    function affect!(integrator)
        integrator.u = Utilities.partition_species(integrator.u)
        integrator.u[length(integrator.u)] = 0
    end
        
    condition(u,t,integrator) = u[length(u)]==N
    cb = DiscreteCallback(condition,affect!)
    sol = solve(jump_prob, SSAStepper(), callback=cb)
    pop!(sol.u)
    pop!(sol.t)
    return sol
end

simulate_stochastic_cycle (generic function with 1 method)

In [93]:
sc2s = @reaction_network begin
        alpha,  0-->m
        beta, m-->p + m
        gamma,  m-->0
        delta, p-->0
end alpha beta gamma delta

Model ##ReactionSystem#350 with 4 equations
States (2):
  m(t)
  p(t)
Parameters (4):
  alpha
  beta
  gamma
  delta

In [94]:
alph = 3.0
beta = 0.03
gamma = 0.2
delta = 0.01
T=100.5


sol = simulate_cycle(sc2s, [alph, beta, gamma, delta], 0.0, 50.0, T,
        [
            round(alph/gamma),
            round(alph*beta/(gamma*delta))
        ]
    )
    
plot(sol, ylabel="Copy number", xlabel="t")
savefig("../../data/trajectories/model_I.svg")

In [95]:
c2s = @reaction_network begin
        alpha,  0-->m
        beta, m-->p + m
        gamma,  m-->0
    end alpha beta gamma

Model ##ReactionSystem#352 with 3 equations
States (2):
  m(t)
  p(t)
Parameters (3):
  alpha
  beta
  gamma

In [96]:
alph = 3.0
beta = 0.1
gamma = 0.2
T=27.5


sol = simulate_cycle(c2s, [alph, beta, gamma], 0.0, 100.0, T,
        [
            round(CanonicalTwoStageModel.mrna(alph, gamma, T, 0)),
            round(CanonicalTwoStageModel.approximate_protein_mean(alph, beta, gamma, T, 0))
        ]
    )
    
plot(sol, ylabel="Copy number", xlabel="t")
savefig("../../data/trajectories/model_II.svg")

In [97]:
mwr = @reaction_network begin
    alpha,  0-->m
    beta, m + r-->p + m + r
    gamma,  m-->0
    rho, 0-->r
end alpha beta gamma rho

alph = 3.0
beta = 0.01
gamma = 0.2
rho = 0.2
T=27.5


sol = simulate_cycle(mwr, [alph, beta, gamma, rho], 0.0, 100.0, T,
        [
            round(ModelWithRibosomes.mrna(alph, gamma, T, 0)),
            round(ModelWithRibosomes.r(rho, T, 0)),
            round(ModelWithRibosomes.p(alph, beta, gamma, rho, T, 0)),
        ]
    )
    
plot(sol, ylabel="Copy number", xlabel="t")
savefig("../../data/trajectories/model_III.svg")

In [98]:
slm = @reaction_network begin
    alpha,  0-->m
    beta, m + r-->p + m + r
    gamma,  m-->0
    rho, 0-->r
    k, 0-->n
end alpha beta gamma rho k

alph = 1.0
beta = 0.01
gamma = 0.2
rho = 1.0
N = 10
T=27.5
k = N/T

sol = simulate_stochastic_cycle(slm, [alph, beta, gamma, rho, k], 0., 100., T,
    [
        round(ModelWithRibosomes.mrna(alph, gamma, T, 0)),
        round(ModelWithRibosomes.r(rho, T, 0)),
        round(ModelWithRibosomes.p(alph, beta, gamma, rho, T, 0)),
        0
    ],
    N)
plot(sol, ylabel="Copy number", xlabel="t")

savefig("../../data/trajectories/model_IV.svg")

In [99]:
slc2sm = @reaction_network begin
    alpha,  0-->m
    beta, m -->p + m
    gamma,  m-->0
    k, 0-->n
end alpha beta gamma k

Model ##ReactionSystem#358 with 4 equations
States (3):
  m(t)
  p(t)
  n(t)
Parameters (4):
  alpha
  beta
  gamma
  k

In [100]:
alph = 1.0
beta = 0.1
gamma = 0.2
N = 10
T=27.5
k = N/T

sol = simulate_stochastic_cycle(slc2sm, [alph, beta, gamma, k], 0., 100., T,
    [
        round(CanonicalTwoStageModel.mrna(alph, gamma, T, 0)),
        round(CanonicalTwoStageModel.approximate_protein_mean(alph, beta, gamma, T, 0)),
        0
    ],
    N)
plot(sol, ylabel="Copy number", xlabel="t")

savefig("../../data/trajectories/model_V.svg")